## 0. 모듈 불러오기

In [ ]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2
import mediapipe as mp

## 1. 데이터 확인

In [ ]:
directory = "C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb"

In [ ]:
Class_name = os.listdir(directory)

In [ ]:
print(Class_name)

In [ ]:
data = []
for i in Class_name:
    path = os.path.join(directory, i)
    file_list = os.listdir(path)
    
    for j in file_list:
        name = j
        label1 = i
        label2 = i[3:5]
        num = j[6:10]
    
        eda = [name, label1, label2, num]
        data.append(eda)

In [ ]:
DF=pd.DataFrame(data, columns=['name','fold','class','num'])
DF['path']=directory+'/'+DF['fold']+'/'+DF['name']
DF

In [ ]:
DF_select=DF[(DF['class']=='15')|(DF['class']=='13')|(DF['class']=='60')|(DF['class']=='01')|(DF['class']=='07')|
             (DF['class']=='55')|(DF['class']=='77')|(DF['class']=='11')|(DF['class']=='16')|(DF['class']=='17')]
DF_select['class']=DF_select['class'].astype(int)
DF_select['num']=DF_select['num'].astype(int)
DF_select

In [ ]:
with open('C:/Users/HOME/Desktop/AI_project/KSL/class_label.p','rb') as f:
    Labels=pickle.load(f)

In [ ]:
DF_label=pd.DataFrame({'class' : Labels.keys(),'label' : Labels.values()})
DF_label

In [ ]:
DF_select = pd.merge(DF_select, DF_label, on='class')
DF_select

In [ ]:
dfdf=DF_select[(DF_select['num']>=31)&(DF_select['num']<=50)]
dfdf

In [ ]:
aa= pd.DataFrame(os.listdir('C:/Users/HOME/Desktop/AI_project/KSL/MP'), columns=['name'])
aa

In [ ]:
bb = pd.merge(aa, DF_select, on='name')
bb

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)
train_generator = datagen.flow_from_dataframe(
          dataframe=bb,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=bb,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

In [ ]:
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow import keras

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
pool_size = 2
    
model1 = Sequential([
  keras.Input(shape=(256,256,3)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),                                           
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Flatten(),
  Dense(50),
  Activation('relu'),                                          
  
  Dense(10, activation='softmax'),
])

model.summary()

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, MaxPooling2D,\
                        Permute, TimeDistributed, Bidirectional,GRU, LeakyReLU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Reshape
from keras import models
from keras import layers
import tensorflow as tf

In [ ]:
Model = Sequential()

Model.add(SeparableConv2D(12,(3,3),activation="relu",
                 input_shape=(256,256,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

#
Model.add(SeparableConv2D(24,(3,3),
                 activation="relu",padding="same"))
Model.add(Dropout(0.2))
Model.add(MaxPooling2D((2,2)))

#
Model.add(TimeDistributed(Flatten()))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))

#
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(10,activation="sigmoid"))

In [ ]:
Model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
checkpoint_filepath = 'C:/Users/HOME/Desktop/AI_project/KSL/h5/' + 'best_model_{val_accuracy:.4f}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [ ]:
#cnn
history1= model1.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

In [ ]:
history2= Model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3), classes=10)

In [ ]:
base_model.trainable = False

In [ ]:
base_model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
history3= base_model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)